В папке subsid находятся файлы (tm_sales_1, tm_sales_2, ...) с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, сджойнив файлы с продажами с логами по подключениям в системе.       
1)Cотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить.       
2)Продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)   
3)Если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается      

In [1]:
import pandas as pd
import os

In [2]:
os.getcwd()

'C:\\Users\\User\\!JupNotebooks'

In [13]:
# Считываем логи:

logs = pd.read_csv(r'./subsid/prod_activations_logs.csv', sep=';')
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43
3,id8641743,2752,20-03-2020 15:44,21-04-2020 15:44
2,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00
6,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42
2,id8741631,5677,19-03-2020 12:28,01-12-2020 00:00


In [14]:
# Считываем продажи из разных файлов в папке и джойним их в один датафрейм:

sales = pd.DataFrame()
path = r'./subsid/'
for i in os.listdir(path):
    if 'sales' in i:
        full_path = path + i
        data1 = pd.read_csv(full_path, sep=';')
        sales = pd.concat([data1,sales], ignore_index = True)

# Cотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то добавим его.Если в 
# файле с продажами встречается строка без указанного SUBS_ID, она пропускается:

sales = sales.dropna(subset=['SUBS_ID'])
sales['SUBS_ID'] = sales['SUBS_ID'].apply(lambda x: 'id'+ x if (x.startswith('id') == False) else x)
sales

,FILIAL_ID,PROD_ID,SUBS_ID,ACT_DTTM
0,3,1499,id5764122,18-03-2020 15:44
1,6,3020,id7642700,15-03-2020 14:21
2,2,5677,id1374509,17-03-2020 11:48
3,3,1743,id5416547,17-03-2020 10:17
5,7,9879,id8362218,05-03-2020 11:42
6,2,3210,id2185490,16-03-2020 16:28
7,1,1954,id4651830,20-03-2020 14:59
8,5,6431,id7646509,19-03-2020 13:00
9,4,3313,id7412683,22-03-2020 17:25


In [5]:
# pd.Timedelta('5 min')

Timedelta('0 days 00:05:00')

In [7]:
# Посчитаем, сколько времени прошло между подключением и отключением:

logs.START_DTTM = pd.to_datetime(logs.START_DTTM)
logs.END_DTTM = pd.to_datetime(logs.END_DTTM)

logs['difference'] = logs.END_DTTM - logs.START_DTTM 
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,difference
1,id4651830,1954,2020-03-20 14:59:00,2020-01-12 00:00:00,-69 days +09:01:00
5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-01-12 00:00:00,-69 days +06:35:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,id8238421,1859,2020-01-03 11:42:00,2020-01-03 11:43:00,0 days 00:01:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,32 days 00:00:00
2,id2185490,3210,2020-03-16 16:28:00,2020-01-12 00:00:00,-65 days +07:32:00
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
2,id8741631,5677,2020-03-19 12:28:00,2020-01-12 00:00:00,-68 days +11:32:00


In [8]:
# Отфильтруем те продажи, для которых отключение (END_DTTM) произошло меньше чем через 5 минут после 
# подключения (START_DTTM)

# logs = logs.query('difference > "5m" or difference < "0m"')
logs = logs[(logs.difference > '5m') | (logs.difference < '0m')]
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,difference
1,id4651830,1954,2020-03-20 14:59:00,2020-01-12 00:00:00,-69 days +09:01:00
2,id7461794,3310,2020-03-20 17:25:00,2020-01-12 00:00:00,-69 days +06:35:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,32 days 00:00:00
2,id2185490,3210,2020-03-16 16:28:00,2020-01-12 00:00:00,-65 days +07:32:00
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
2,id8741631,5677,2020-03-19 12:28:00,2020-01-12 00:00:00,-68 days +11:32:00


In [9]:
# Найдем те id, которые удовлетворяют всем нашим требованиям (корректные подключения):

data = sales.merge(logs)
data = data.sort_values('SUBS_ID')
data

,FILIAL_ID,PROD_ID,SUBS_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
2,2,3210,id2185490,16-03-2020 16:28,2020-03-16 16:28:00,2020-01-12 00:00:00,-65 days +07:32:00
3,1,1954,id4651830,20-03-2020 14:59,2020-03-20 14:59:00,2020-01-12 00:00:00,-69 days +09:01:00
1,3,1743,id5416547,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
0,6,3020,id7642700,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [10]:
data.SUBS_ID.str.cat(sep=', ')

'id2185490, id4651830, id5416547, id7642700'

In [69]:
# Запишем корректные подключения в датафрэйм с разделителем ;.
# data[['ACT_DTTM', 'FILIAL_ID', 'difference']].to_csv('prod_activations',sep=';',index=False)